In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor 
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error as MAERROR
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.model_selection import GroupShuffleSplit

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import callbacks


In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
hp = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
hppred = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')

In [ ]:
# Have a glimpse of the data
hp.head()

# Shape

In [ ]:
# what is the shape
print(hp.shape)
print(hppred.shape)
# there are 1460 rows and 81 columns in training data
# (1459,80) in testing data. So one column is missing in test data.

# so there NANs. Are there missing values?

In [ ]:
hp.isna().head() # isnull() also works

In [ ]:
hp.isnull().sum().sum()

In [ ]:
hp.isnull().sum()

In [ ]:
colnasum_tr = hp.isnull().sum().sort_values(ascending=False) 
colnasum_pr = hppred.isnull().sum().sort_values(ascending=False) 
print(colnasum_tr)
print(colnasum_pr)
# We get a Series with index as column names and values are number of NANs or missing values

In [ ]:
# We can plot the above Series.Pandas has elementary ploting 
hp.isnull().sum().plot() # 


In [ ]:
# we can plot just a few values of the Series
hp.isnull().sum().iloc[0:20].plot()

In [ ]:
hppred.isnull().sum().plot()

# what are the columns with NANs 

In [ ]:
colna = colnasum_tr[colnasum_tr >0]
print(colna)
colna.shape
# so there are 19 columns having NANs

# Let us first try to analyze by dropping the columns with NANs

In [ ]:
hp_dna = hp.dropna(axis=1)
hppred_dna = hppred.dropna(axis=1)

hp_dna

In [ ]:
# check no missing values
hp_dna.isnull().sum().sum()
# No NANs or missing values

# Identify the target column 

In [ ]:
# first list the columns
cols = hp_dna.columns
# find out columns with 'Sales' in their names
colsale = [col for col in cols if 'Sale' in col]
colsale
# Of ['SaleType', 'SaleCondition', 'SalePrice'] 'SalePrice' is our target variable

In [ ]:
# first list the columns
colspr = hppred_dna.columns
# find out columns with 'Sales' in their names
colsale = [col for col in colspr if 'Sale' in col]
colsale
# oops! prediction data has no 'SalePrice' and 'SaleType' columns


# So the columns in hptest_dna and hp_dna might be different. So first we got to figure out which columns match and which don't

In [ ]:
pd.Series(cols).equals(colspr)
# Output 'False' implies both have different columns

In [ ]:
# print columns which match in both
colsmatch = [col2 for col1 in cols for col2 in colspr if col1 == col2]
print(colsmatch)
print('')
print('NUmber of matching columns=', len(colsmatch))

# Number of matching columns in both are 46. So form another training and testing sets with just these matching forms.

In [ ]:
hp_dna_X = hp_dna[colsmatch]
hp_dna_prX = hppred_dna[colsmatch]
hp_y = hp_dna['SalePrice']

# Split the data into training and testing data

In [ ]:
# Split into validation and training data
hptrain_X, hpval_X, hptrain_y, hpval_y = train_test_split(hp_dna_X, hp_y, random_state=2)
hptrain_X.reset_index(inplace=True) 
hptrain_X.pop('index')
hpval_X.reset_index(inplace=True)
hpval_X.pop('index')
hpval_X.head() 
#train_y= hptrain_y.reset_index()
#train_y.pop('index')
#hptrain_y

# above things are done because the output of test-train-split will have random index values. So that index
# must be removed and a new index is placed

In [ ]:
# Let us first work with numeric columns only
Xtr_int = hptrain_X.select_dtypes(include = int)
Xval_int = hpval_X.select_dtypes(include = int)
Xpr_int = hp_dna_prX.select_dtypes(include = int)


# Let's use the Random Forest regressor 

In [ ]:
RFmodel = RandomForestRegressor(random_state=100)

# Fit the model

In [ ]:
RFmodel.fit(Xtr_int,hptrain_y)

# Compute the MAE for Xtr_int

In [ ]:
Xtr_int_mae = MAERROR(RFmodel.predict(Xtr_int),hptrain_y)
print(Xtr_int_mae)

# Compute the MAE for Xval_int

In [ ]:
pred_valy = RFmodel.predict(Xval_int)
Xval_int_mae = MAERROR(pred_valy,hpval_y)
print(Xval_int_mae)

In [ ]:
# Make predictions on test data
pred_yrf = RFmodel.predict(Xpr_int)
pred_yrf.shape

# Refine the results by exploring the data and refining the model. 

#  Refinement-1: Let us add categorical variables as well into analysis

In [ ]:
hptrain_X.select_dtypes(object).shape

In [ ]:
# check which columns have 'object' data types
objs = (hptrain_X.dtypes == 'object')
# print out names of object columns
print(objs[objs],'\n')

# make a list of those columns
ObjCols = list(objs[objs].index)
#print(ObjCols,'\n')

# Check that the same list of 'objcols' works for the testing data 
objs2 = (hp_dna_prX.dtypes == 'object')
ObjCols2 = np.array(list(objs2[objs2].index))
#print(ObjCols2,'\n')
print(np.array_equal(ObjCols2,ObjCols)) 

# Because of the finiteness of the sample data, the number of unique variables per a object column are different between the training, validation and test samples as the next cell shows

In [ ]:
print(hpval_X[ObjCols].nunique().sum(),hptrain_X[ObjCols].nunique().sum(),hp_dna_prX[ObjCols].nunique().sum())

## Therefore, retain only those object columns which have same number of unique values for the categorical variables in all data samples

In [ ]:
stmt1 = hptrain_X[ObjCols].nunique() == hpval_X[ObjCols].nunique() 
stmt2 = hptrain_X[ObjCols].nunique() == hp_dna_prX[ObjCols].nunique()
stmt3 = (stmt1) & (stmt2)
ObjColsCmn = list(stmt3[stmt3].index)
print('\n','Object columns that have same nunique() values in \
      train,validate and test samples are',ObjColsCmn,'\n')
# Now form a dataframe of above object columns for training, validation and test sets 
Xtrain = hptrain_X[ObjColsCmn]
Xval = hpval_X[ObjColsCmn]
Xpr = hp_dna_prX[ObjColsCmn]


#drop the categorical columns
XtrainObjDrop= hptrain_X.drop(ObjCols,axis=1)
XvalObjDrop= hpval_X.drop(ObjCols,axis=1)
XprObjDrop= hp_dna_prX.drop(ObjCols,axis=1)

#concat the dataframes(hereafter call DF) with OneHotEncoded columns
Xtrain = pd.concat([XtrainObjDrop,Xtrain],axis =1)
Xvalid = pd.concat([XvalObjDrop,Xval],axis =1)
Xpred = pd.concat([XprObjDrop,Xpr],axis =1)

#print shape
print('Shape of training set is',XtrainObjDrop.shape)
print('')
print(hpval_X.shape,hptrain_X.shape)
print('')
print(Xtrain.shape,Xvalid.shape,Xpred.shape)

# List categorical and numerical features
CatCol = list(Xtrain.select_dtypes('object').columns)
NumCol = list(Xtrain.select_dtypes(['int64','float64']).columns)
print('')
print('List of categorical columns is',CatCol)
print('')
print('List of Numerical columns is',NumCol)

# Use preprocessor for scaling and one hot encoding
preprocessor = make_column_transformer(
    (StandardScaler(), NumCol),
    (OneHotEncoder(), CatCol),
)

Xtrain = preprocessor.fit_transform(Xtrain)
Xvalid = preprocessor.fit_transform(Xvalid)
Xpred = preprocessor.fit_transform(Xpred)

input_shape = [Xtrain.shape[1]]
print('')
print("Input shape: {}".format(input_shape))

In [ ]:
hptrain_y.min(),hpval_y.min(),hptrain_y.max(),hpval_y.max()

In [ ]:
hptrain_y = (hptrain_y)/1000

In [ ]:
hpval_y = (hpval_y)/1000

 ## Construct the neural net with the tensorflow 

In [ ]:
from tensorflow.keras import callbacks

# YOUR CODE HERE: define an early stopping callback
early_stopping = callbacks.EarlyStopping(
    min_delta = 0.5,
    patience = 100,
    restore_best_weights = True,
)

In [ ]:
# 
model = keras.Sequential([
    layers.BatchNormalization(input_shape=input_shape),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.3),
    layers.BatchNormalization(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.3),
    layers.BatchNormalization(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.3),
    layers.BatchNormalization(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.3),
    layers.BatchNormalization(),
    layers.Dense(512, activation='relu'),
    layers.Dropout(0.3),
    layers.BatchNormalization(),
    layers.Dense(512, activation='relu'),
    layers.Dense(1)
])
model.compile(
    optimizer='adam',
    loss='mae',
)
history = model.fit(
    Xtrain, hptrain_y,
    validation_data=(Xvalid, hpval_y),
    batch_size=200,
    epochs=1000,
    verbose = 0,
    callbacks=[early_stopping]
)
history_df = pd.DataFrame(history.history)
history_df.loc[:, ['loss', 'val_loss']].plot()
print("Minimum Validation Loss: {:0.4f}".format(history_df['val_loss'].min()));

# Compute MAE for Xtrain

In [ ]:
Xtrain_mae = MAERROR(model.predict(Xtrain),hptrain_y)
print(Xtrain_mae)

# Compute MAE for Xvalid

In [ ]:

Xval_mae = MAERROR(model.predict(Xvalid),hpval_y)
print(Xval_mae)

# Predict 'SalesPrice' for testing data and also get the MAE

In [ ]:
pred_y = model.predict(Xpred)
pred_y.shape

In [ ]:
pred_y[:,0]-pred_yrf

In [ ]:
output = pd.DataFrame({'Id': hp_dna_prX['Id'],
                       'SalePrice': pred_y[:,0]*1000})
output.to_csv('submission.csv', index=False)

output2 = pd.DataFrame({'Id': hp_dna_prX['Id'],
                       'SalePrice': pred_y[:,0]*1000,'SalePrice-rf': pred_yrf })
output2.to_csv('saleprice-nn-rf.csv', index=False)


In [ ]:
pd.read_csv('saleprice-nn-rf.csv')